In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.6 MB/s eta 0:00:00


In [3]:
!pip install weightwatcher

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 1.4 MB/s eta 0:00:00


In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=c8ab43c62e7bdbaa67428411286a2f94474d6c84f0b0cb7d9af323ffa889a862
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.1 MB/s eta 0:00:00


In [6]:
import torch
import numpy as np
import weightwatcher as ww
import argparse
import os
from transformers import BertModel, BertPreTrainedModel
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from transformers import get_scheduler
from accelerate import Accelerator
import accelerate.utils


In [7]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


In [8]:
watcher = ww.WeightWatcher(model=model)

In [9]:
details= watcher.analyze()

In [10]:
summary = watcher.get_summary(details)

In [11]:
print(summary['alpha'], summary['alpha_weighted'])

4.128993187239748 5.628037969921715


In [12]:
print(details)

    layer_id       name         D    M      N          Q      alpha  \
0          2  Embedding  0.073591  768  30522  39.742188   3.026795   
1          3  Embedding  0.100017  512    768   1.500000   1.713530   
2         12     Linear  0.041366  768    768   1.000000   2.892591   
3         13     Linear  0.046417  768    768   1.000000   3.205469   
4         14     Linear  0.091982  768    768   1.000000   4.257153   
..       ...        ...       ...  ...    ...        ...        ...   
70       212     Linear  0.104283  768    768   1.000000  14.356094   
71       215     Linear  0.080539  768    768   1.000000   8.978272   
72       219     Linear  0.022538  768   3072   4.000000   3.884860   
73       222     Linear  0.046172  768   3072   4.000000   6.908946   
74       226     Linear  0.036749  768    768   1.000000   2.033796   

    alpha_weighted   entropy  has_esd  ...     sigma spectral_norm  \
0        13.470843  0.663233     True  ...  0.107571  28218.226562   
1      

In [13]:
print(summary)

{'log_norm': 3.1325607, 'alpha': 4.128993187239748, 'alpha_weighted': 5.628037969921715, 'log_alpha_norm': 6.069402945814816, 'log_spectral_norm': 1.4700447, 'stable_rank': 56.408802}


In [14]:
model = BertModel.from_pretrained('bert-base-uncased')


In [15]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [16]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [18]:
print(str(details['longname']))

0                  embeddings.word_embeddings
1              embeddings.position_embeddings
2        encoder.layer.0.attention.self.query
3          encoder.layer.0.attention.self.key
4        encoder.layer.0.attention.self.value
                       ...                   
70      encoder.layer.11.attention.self.value
71    encoder.layer.11.attention.output.dense
72        encoder.layer.11.intermediate.dense
73              encoder.layer.11.output.dense
74                               pooler.dense
Name: longname, Length: 75, dtype: object


In [19]:
filtered_tables = details[details['longname'].str.contains("attention|embeddings")==False]
module_to_train = filtered_tables.sort_values(by=['alpha'], ascending=True)['longname'].iloc[:10].to_list()
print("module_to_train", module_to_train)

module_to_train ['pooler.dense', 'encoder.layer.1.intermediate.dense', 'encoder.layer.2.intermediate.dense', 'encoder.layer.3.intermediate.dense', 'encoder.layer.0.intermediate.dense', 'encoder.layer.5.output.dense', 'encoder.layer.4.output.dense', 'encoder.layer.4.intermediate.dense', 'encoder.layer.2.output.dense', 'encoder.layer.0.output.dense']


In [20]:
print(filtered_tables)

    layer_id    name         D    M     N    Q     alpha  alpha_weighted  \
6         21  Linear  0.058660  768  3072  4.0  3.364346        7.452674   
7         24  Linear  0.036427  768  3072  4.0  3.559378        6.087193   
12        39  Linear  0.045781  768  3072  4.0  2.703059        5.925205   
13        42  Linear  0.029702  768  3072  4.0  3.835272        7.358685   
18        57  Linear  0.030629  768  3072  4.0  3.039848        6.674360   
19        60  Linear  0.019480  768  3072  4.0  3.500975        6.391030   
24        75  Linear  0.014151  768  3072  4.0  3.245552        7.140780   
25        78  Linear  0.021216  768  3072  4.0  3.563441        7.312189   
30        93  Linear  0.015341  768  3072  4.0  3.474146        7.627374   
31        96  Linear  0.025949  768  3072  4.0  3.444712        6.734910   
36       111  Linear  0.023730  768  3072  4.0  3.582461        7.750578   
37       114  Linear  0.040728  768  3072  4.0  3.374800        6.266069   
42       129

In [21]:
print(details['longname'])

0                  embeddings.word_embeddings
1              embeddings.position_embeddings
2        encoder.layer.0.attention.self.query
3          encoder.layer.0.attention.self.key
4        encoder.layer.0.attention.self.value
                       ...                   
70      encoder.layer.11.attention.self.value
71    encoder.layer.11.attention.output.dense
72        encoder.layer.11.intermediate.dense
73              encoder.layer.11.output.dense
74                               pooler.dense
Name: longname, Length: 75, dtype: object


In [22]:
from typing import Optional, Tuple, Union
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutput
class BERTFT(BertPreTrainedModel):  # Done
    """
    BERT Fine-Tuning on GLUE tasks
    """

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout
            if config.classifier_dropout is not None
            else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.new_layer = nn.Linear(self.bert.config.hidden_size, config.hidden_size)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.relu = nn.ReLU()
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        Feed-forward function for the bert ft model
        """
        return_dict = (
            return_dict if return_dict is not None else self.config.use_return_dict
        )

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        intermediate_output = self.new_layer(pooled_output)
        intermediate_output = self.relu(intermediate_output)
        intermediate_output = self.dropout(intermediate_output)
        logits = self.classifier(intermediate_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    #  We are doing regression
                    self.config.problem_type = "regression"
                elif (
                    self.num_labels > 1
                    and labels.dtype == torch.long
                    or labels.dtype == torch.int
                ):
                    # We are doing multi-class classification
                    self.config.problem_type = "single_label_classification"
                else:
                    # We are doing multi-label classification
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [23]:
model = BERTFT.from_pretrained('bert-base-uncased',
                               )

Some weights of BERTFT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'new_layer.bias', 'new_layer.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model

BERTFT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [25]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00


In [26]:
import datasets
import pandas as pd
from transformers import BertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
import random
from transformers import DataCollatorWithPadding
from transformers import get_scheduler

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    SchedulerType,
    default_data_collator,
    get_scheduler,
)

In [27]:
task_to_keys = {  # Done
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [28]:
def get_train_eval(task_name, model_name, pad_to_max_length=False, max_length=512, batch_size=32, slow_tokenizer=False):  # WORK IN PROGRESS

    num_labels = 1

    # Load Raw Data and find num_labels
    if task_name is not None:
        raw_datasets = load_dataset("glue", task_name)
        is_regression = task_name == "stsb"
        if not is_regression:
            label_list = raw_datasets["train"].features["label"].names  # type: ignore
            num_labels = len(label_list)
    else:
        raw_datasets = load_dataset("glue", "all")
        is_regression = raw_datasets["train"].features["label"].dtype in ["float32", "float64"]  # type: ignore
        if not is_regression:
            label_list = raw_datasets["train"].unique("label")  # type: ignore
            label_list.sort()
            num_labels = len(label_list)

    # Load Tokenizer
    tokenizer = BertTokenizer.from_pretrained( # Done
        model_name,
        do_lower_case=True,
        use_fast=not slow_tokenizer,
    )

    # Define keys for both inputs
    if task_name is not None:
        sentence1_key, sentence2_key = task_to_keys[task_name]
    else:
        sentence1_key, sentence2_key = "sentence1", "sentence2"

    # Set target padding
    padding = "max_length" if pad_to_max_length else False

    # Preprocess Data
    def preprocess(input):
        texts = (
            (input[sentence1_key],)
            if sentence2_key is None
            else
            (input[sentence1_key], input[sentence2_key])
        )
        result = tokenizer(
            *texts, padding=padding,
            max_length=max_length,
            truncation=True
        )
        return result

    processed_datasets = raw_datasets.map(
        preprocess,
        batched=True,
        remove_columns=raw_datasets["train"].column_names,  # type: ignore
        # desc="Running tokenizer on dataset",
    )

    train_dataset = processed_datasets["train"]  # type: ignore
    eval_dataset = processed_datasets["validation_matched" # type: ignore
                                      if task_name == "mnli"
                                      else "validation"]

    for index in random.sample(range(len(train_dataset)), 3):
        print(f"Sample {index} of train set: {train_dataset[index]}")

    if pad_to_max_length:
        data_collator = default_data_collator
    else:
        data_collator = DataCollatorWithPadding(tokenizer,
                                                pad_to_multiple_of=8)

    train_dataloader = DataLoader(
        train_dataset,  # type: ignore
        shuffle=True,
        collate_fn=data_collator,
        batch_size=batch_size,
    )

    eval_dataloader = DataLoader(
        eval_dataset,  # type: ignore
        collate_fn=data_collator,
        batch_size=batch_size,
    )

    return num_labels, train_dataloader, eval_dataloader

In [29]:
num_labels, train_dataloader, eval_dataloader = get_train_eval(model_name='bert-base-uncased',task_name='cola')

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

Sample 4243 of train set: {'input_ids': [101, 2008, 3899, 2003, 2061, 27863, 1010, 2009, 2130, 2699, 2000, 6805, 2370, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Sample 7140 of train set: {'input_ids': [101, 3021, 2097, 2404, 1037, 3861, 1997, 2016, 2014, 2006, 2115, 4624, 2077, 4826, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Sample 2259 of train set: {'input_ids': [101, 1996, 25516, 19223, 1996, 3011, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [30]:
print(f'Training data size: {len(train_dataloader)}')
print(f'Validation data size: {len(eval_dataloader)}')

Training data size: 268
Validation data size: 33


In [31]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters()],
            "weight_decay": 0.0,
        },
    ]

In [33]:
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=2e-5)

In [34]:
accelerator = Accelerator()

In [35]:
lr_scheduler = get_scheduler(
        name='linear',
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=1,
    )

In [36]:
device = None
if(torch.cuda.is_available()):
  device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [37]:
def get_data(task_name='mnli',pad_to_max_length=False, max_length=512, per_device_train_batch_size=8, per_device_eval_batch_size=8):
  if task_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset("glue", task_name)
# Labels
  if task_name is not None:
    is_regression = task_name == "stsb"
    if not is_regression:
      label_list = raw_datasets["train"].features["label"].names
      num_labels = len(label_list)
    else:
      num_labels = 1
  else:
    # Trying to have good defaults here, don't hesitate to tweak to your needs.
    is_regression = raw_datasets["train"].features["label"].dtype in ["float32", "float64"]
    if is_regression:
      num_labels = 1
    else:
      # A useful fast method:
      # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.unique
      label_list = raw_datasets["train"].unique("label")
      label_list.sort()  # Let's sort it for determinism
      num_labels = len(label_list)
  config = AutoConfig.from_pretrained('bert-base-uncased', num_labels=num_labels, finetuning_task=task_name)
  tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)
 # Preprocessing the datasets
  print(label_list)

  if task_name is not None:
    sentence1_key, sentence2_key = task_to_keys[task_name]
  else:
    # Again, we try to have some nice defaults but don't hesitate to tweak to your use case.
    non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
    if "sentence1" in non_label_column_names and "sentence2" in non_label_column_names:
      sentence1_key, sentence2_key = "sentence1", "sentence2"
    else:
      if len(non_label_column_names) >= 2:
        sentence1_key, sentence2_key = non_label_column_names[:2]
      else:
        sentence1_key, sentence2_key = non_label_column_names[0], None

  # Some models have set the order of the labels to use, so let's make sure we do use it.
  label_to_id = None
  if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id
    and task_name is not None
    and not is_regression):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        print(
            f"The configuration of the model provided the following label correspondence: {label_name_to_id}. "
            "Using it!"
        )
        label_to_id = {i: label_name_to_id[label_list[i]] for i in range(num_labels)}
    else:
        print(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",
        )
  elif task_name is None and not is_regression:
      label_to_id = {v: i for i, v in enumerate(label_list)}

  if label_to_id is not None:
      model.config.label2id = label_to_id
      model.config.id2label = {id: label for label, id in config.label2id.items()}
  elif task_name is not None and not is_regression:
      model.config.label2id = {l: i for i, l in enumerate(label_list)}
      model.config.id2label = {id: label for label, id in config.label2id.items()}

  padding = "max_length" if pad_to_max_length else False

  def preprocess_function(examples):
      # Tokenize the texts
      texts = (
          (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
      )
      result = tokenizer(*texts, padding=padding, max_length=max_length, truncation=True)

      if "label" in examples:
          if label_to_id is not None:
              # Map labels to IDs (not necessary for GLUE tasks)
              result["labels"] = [label_to_id[l] for l in examples["label"]]
          else:
              # In all cases, rename the column to labels because the model will expect that.
              result["labels"] = examples["label"]
      return result

  with accelerator.main_process_first():
      processed_datasets = raw_datasets.map(
          preprocess_function,
          batched=True,
          remove_columns=raw_datasets["train"].column_names,
          desc="Running tokenizer on dataset",
      )

  train_dataset = processed_datasets["train"]
  eval_dataset = processed_datasets["validation_matched" if task_name == "mnli" else "validation"]

  # Log a few random samples from the training set:
  for index in random.sample(range(len(train_dataset)), 3):
      print(f"Sample {index} of the training set: {train_dataset[index]}.")

  # DataLoaders creation:
  if pad_to_max_length:
      # If padding was already done ot max length, we use the default data collator that will just convert everything
      # to tensors.
      data_collator = default_data_collator
  else:
      # Otherwise, `DataCollatorWithPadding` will apply dynamic padding for us (by padding to the maximum length of
      # the samples passed). When using mixed precision, we add `pad_to_multiple_of=8` to pad all tensors to multiple
      # of 8s, which will enable the use of Tensor Cores on NVIDIA hardware with compute capability >= 7.5 (Volta).
      data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None))

  train_dataloader = DataLoader(
      train_dataset, shuffle=True, collate_fn=data_collator, batch_size=per_device_train_batch_size
  )
  eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=per_device_eval_batch_size)
  return num_labels, train_dataloader, eval_dataloader

In [110]:
num_labels, train_dataloader, eval_dataloader = get_data(task_name='cola')

Running tokenizer on dataset:   0%|          | 0/8551 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1043 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/1063 [00:00<?, ? examples/s]

Sample 1491 of the training set: {'input_ids': [101, 2008, 1037, 3319, 2234, 2041, 7483, 2003, 23546, 1997, 2023, 3720, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 0}.
Sample 7519 of the training set: {'input_ids': [101, 1996, 3337, 4061, 2007, 2169, 2060, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 1}.
Sample 602 of the training set: {'input_ids': [101, 1996, 14366, 15748, 4064, 1997, 2300, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 1}.


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:516: FutureWarning: The `use_fp16` property is deprecated and will be removed in version 1.0 of Accelerate use `Accelerator.mixed_precision == 'fp16'` instead.
  warnings.warn(


In [111]:
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
    )

In [116]:
# task_name = "mrpc"
# dataset = load_dataset("glue", task_name)
# train_data = dataset["train"]
# eval_data = dataset["validation"]

# print(train_data)

loss = 0
val_examples = 0
correct = 0
model.eval()
for step, batch in enumerate(train_dataloader):
    """
    Assuming batch = (input_ids, attention_mask, labels)
    """
    batch = batch.to(device)
    input_len = len(batch)
    with torch.no_grad():
        outputs = model(
            **batch
        )
        logits = outputs.logits
        _, predict = torch.max(logits, dim=1)
        correct += sum(predict == batch['labels']).item()  # type: ignore
        eloss = outputs.loss
        loss += eloss.item() * input_len
        val_examples += input_len
print(loss / val_examples, correct / val_examples)

0.7362300788163023 0.6414873713751169
